In [ ]:
import torch
from torch import autocast
from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline
import requests
from PIL import Image
from io import BytesIO

### Text2Image flow

In [ ]:

#model_id = "CompVis/stable-diffusion-v1-4"
model_id = "runwayml/stable-diffusion-v1-5"
device = "cuda"

# this for the regular FP32 model
#pipe = StableDiffusionPipeline.from_pretrained(model_id, use_auth_token=True)
#pipe = pipe.to(device)

#this for the FP16 model
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16, revision="fp16", use_auth_token=True)
pipe = pipe.to(device)

In [ ]:
prompt = "a photo of an astronaut riding a horse on mars"
batch_size = 1
prompt_input = [prompt] * batch_size

with autocast("cuda"):
    image = pipe(prompt_input, guidance_scale=7.5).images[0]
    
#image.save("astronaut_rides_horse.png")

image

In [ ]:
# this is to test out how to convert to base64, to pass in using to the managed endpoint
from io import BytesIO
import base64

result_image_str = [""] * 3
print(len(result_image_str))

for i in range(len(images)):

    buffered_result_image = BytesIO()

    images[i].save(buffered_result_image, format="PNG")
    result_image_str[i] = base64.b64encode(buffered_result_image.getvalue()).decode('utf-8')

result_image_str

In [ ]:
#this is how you save the model to register on the aml workspace
pipe.save_pretrained('models/stable-diffusion-v1-5')

### Image2Image Flow

In [ ]:
model_id = "CompVis/stable-diffusion-v1-4"
#model_id = "runwayml/stable-diffusion-v1-5"
device = "cuda"


#this for the FP16 model
pipe = StableDiffusionImg2ImgPipeline.from_pretrained(model_id, torch_dtype=torch.float16, revision="fp16", use_auth_token=True)
pipe = pipe.to(device)

In [ ]:
from PIL import Image
import os

file =  os.path.join("images", "elmo.jpg")
img = Image.open(file).convert("RGB")
img = img.resize((512, 512))

In [ ]:
prompt = "a photo of elmo stangind in front of a fiery pit"

with autocast("cuda"):
    image = pipe(prompt=prompt, image=img, strength=.75, guidance_scale=7.5).images[0]
    
image

#image.save("astronaut_rides_horse.png")